In [ ]:

from google.colab import files
uploaded = files.upload()  

import zipfile
import os
from tqdm import tqdm  


zip_path = list(uploaded.keys())[0]
extract_folder = 'dataset'

os.makedirs(extract_folder, exist_ok=True)


with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    file_list = zip_ref.namelist()
    for file in tqdm(file_list, desc="Extracting files"):
        zip_ref.extract(file, extract_folder)

print(f"Dataset extracted to '{extract_folder}' folder successfully!")


Saving road damage det.v4i.yolov8.zip to road damage det.v4i.yolov8.zip


Extracting files: 100%|██████████| 18788/18788 [00:02<00:00, 8870.91it/s] 

Dataset extracted to 'dataset' folder successfully!


In [2]:
!pip install ultralytics opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.1 MB/s eta 0:00:00


In [ ]:

from ultralytics import YOLO


data_yaml = "PATH_TO_YAML_FILE"

model = YOLO("/content/yolov8m.pt")


model.train(
    data=data_yaml,
    epochs=50,
    imgsz=640,
    batch=16,

    name="road_damage_yolov8m"
)


metrics = model.val()
print(metrics)




Ultralytics 8.3.206 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=road_damage_yolov8, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0

ValueError: Model is already in PyTorch format. Valid formats are ('torchscript', 'onnx', 'openvino', 'engine', 'coreml', 'saved_model', 'pb', 'tflite', 'edgetpu', 'tfjs', 'paddle', 'mnn', 'ncnn', 'imx', 'rknn')

In [1]:


import os
import matplotlib.pyplot as plt
from ultralytics import YOLO
import seaborn as sns
import numpy as np

# Load the best trained model
model = YOLO("weights/best.pt")

metrics = model.val()
print(metrics)  

results_dir = "/content/runs/train/road_damage_yolov8m"

# Training results are stored in results.png and results.csv
results_img_path = os.path.join(results_dir, "results.png")

if os.path.exists(results_img_path):
    from IPython.display import Image, display
    print("\nTraining Metrics Visualization:")
    display(Image(filename=results_img_path))
else:
    print("No results.png found. Maybe training folder differs.")



conf_matrix_path = os.path.join(results_dir, "confusion_matrix.png")
if os.path.exists(conf_matrix_path):
    print("\nConfusion Matrix:")
    display(Image(filename=conf_matrix_path))
else:
    print("No confusion_matrix.png found. Generating manually...")

   
    results = model.val(save_json=True)
    cm = results.confusion_matrix.matrix 
    labels = list(model.names.values())

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt=".0f", cmap="Blues",
                xticklabels=labels, yticklabels=labels)
    plt.xlabel("Predicted Class")
    plt.ylabel("True Class")
    plt.title("Confusion Matrix")
    plt.show()


print("\nClass-wise Evaluation Metrics:")
results = model.val()
names = model.names

# Each class metric
for i, name in names.items():
    prec = results.class_result(i)[0]
    rec = results.class_result(i)[1]
    map50 = results.class_result(i)[2]
    map95 = results.class_result(i)[3]
    print(f"Class: {name}\n  Precision: {prec:.3f}\n  Recall: {rec:.3f}\n  mAP@50: {map50:.3f}\n  mAP@50-95: {map95:.3f}\n")


pr_curve_path = os.path.join(results_dir, "PR_curve.png")
f1_curve_path = os.path.join(results_dir, "F1_curve.png")

if os.path.exists(pr_curve_path):
    print("\nPrecision–Recall Curve:")
    display(Image(filename=pr_curve_path))

if os.path.exists(f1_curve_path):
    print("\nF1-Confidence Curve:")
    display(Image(filename=f1_curve_path))

print("\nModel Evaluation Complete! All visualizations generated.")


Ultralytics 8.3.80  Python-3.10.0 torch-2.6.0+cpu CPU (13th Gen Intel Core(TM) i5-13450HX)
Model summary (fused): 92 layers, 25,843,234 parameters, 0 gradients, 78.7 GFLOPs


FileNotFoundError: '/content/dataset/data.yaml' does not exist